Idea: 

For each container, say multiplier is $x$, number of inhabitants is $y$, let $z$% of players choose this chest

pnl = x / (y+z)

First assume uniform distribution of players

In [1]:
# List of tuples: (multiplier, inhabitants)
import numpy as np
player_density = np.array([0.1] * 10) # uniform density
containers_data = np.array([
    (10, 1),
    (80, 6),
    (37, 3),
    (17, 1),
    (31, 2),
    (90, 10),
    (50, 4),
    (20, 2),
    (73, 4),
    (89, 8)
])

dic = {}
for i, (x,y) in enumerate(containers_data):
    pnl = x / (y + player_density[i] * 100)
    if x not in dic:
        dic[x] = round(pnl, 4)
    else:
        dic[x] += round(pnl, 4)

print(dic)

{10: 0.9091, 80: 5.0, 37: 2.8462, 17: 1.5455, 31: 2.5833, 90: 4.5, 50: 3.5714, 20: 1.6667, 73: 5.2143, 89: 4.9444}


Here we can see (73,4) chest is best, with 52k profits, followed by (80,6)chest with 50k profits, followed by (89,8) with 49k profits

Assuming player distributino is equal to pnl distribution:

In [2]:
pnls = np.array(list(dic.values()))
pnls

array([0.9091, 5.    , 2.8462, 1.5455, 2.5833, 4.5   , 3.5714, 1.6667,
       5.2143, 4.9444])

In [3]:
player_density = pnls / np.sum(pnls)
print('player percentage under uniform PnL:', player_density * 100)
dic = {}
for i, (x,y) in enumerate(containers_data):
    pnl = x / (y + player_density[i] * 100)
    if x not in dic:
        dic[x] = round(pnl, 4)
    else:
        dic[x] += round(pnl, 4)

print(dic)
key_max = max(dic, key=dic.get)
print('best chester:', key_max, dic[key_max])

player percentage under uniform PnL: [ 2.77326126 15.2527844   8.68249499  4.71463566  7.88050359 13.72750596
 10.89475884  5.08436315 15.90651873 15.08317343]
{10: 2.6502, 80: 3.7642, 37: 3.1671, 17: 2.9748, 31: 3.1375, 90: 3.7931, 50: 3.3569, 20: 2.8231, 73: 3.6671, 89: 3.8556}
best chester: 89 3.8556


In this case, (89,8) is best with 38k profits, followed by 
(90,10) with 37.9k, (80,6) with 37.6, and (73,4) with 36.6k

In this case, no chest is worth 50k, so we should only open one chest.

Some of the good chests are (89,8), (90,10), (80,6) and (73,4).

Chests like (17,1) or (31,2) provide good profits if too many players choose the best 4 chests.

But these two are very vulnerable, as even if just 10% of players choose either of these two,
we will be left with 17/11, 31/12, or 15k and 25k profits


Again assume player distribution roughly follows pnl distribution with uniform assumption

We then add N(0, 0.1) noise to each chest, but make sure density still sums up to 1

And simulate this for 100k times.

In [4]:
def add_noise(player_density, sigma=0.1):
    noise = np.random.normal(0, sigma, size=player_density.shape)

    # Ensure non-negative
    noisy_density = np.clip(player_density + noise, 0, None) 

    # Renormalize to sum to 1
    normalized_density = noisy_density / np.sum(noisy_density)
    return normalized_density

N_sim = 100000
dic = {}
for _ in range(N_sim):
    noise_density = add_noise(player_density)
    for i, (x,y) in enumerate(containers_data):
        pnl = x / (y + noise_density[i] * 100)
        if x not in dic:
            dic[x] = round(pnl, 4)
        else:
            dic[x] += round(pnl, 4)
for keys in dic.keys():
    dic[keys] = round(dic[keys] / N_sim, 4)
print(dic)
key_max = max(dic, key=dic.get)
print('best chester:', key_max, dic[key_max])



{10: 5.0556, 80: 5.0372, 37: 5.2429, 17: 7.393, 31: 6.1494, 90: 4.5923, 50: 5.1021, 20: 4.8927, 73: 5.377, 89: 4.8519}
best chester: 17 7.393


In [5]:
dic = {}
for _ in range(N_sim):
    noise_density = add_noise(player_density, sigma=0.2)
    for i, (x,y) in enumerate(containers_data):
        pnl = x / (y + noise_density[i] * 100)
        if x not in dic:
            dic[x] = round(pnl, 4)
        else:
            dic[x] += round(pnl, 4)
for keys in dic.keys():
    dic[keys] = round(dic[keys] / N_sim, 4)
print(dic)
key_max = max(dic, key=dic.get)
print('best chester:', key_max, dic[key_max])

{10: 5.2401, 80: 6.5501, 37: 6.235, 17: 8.3366, 31: 7.4334, 90: 5.3201, 50: 6.3023, 20: 5.32, 73: 7.7592, 89: 5.9792}
best chester: 17 8.3366


In [6]:
dic = {}
for _ in range(N_sim):
    noise_density = add_noise(player_density, sigma=0.05)
    for i, (x,y) in enumerate(containers_data):
        pnl = x / (y + noise_density[i] * 100)
        if x not in dic:
            dic[x] = round(pnl, 4)
        else:
            dic[x] += round(pnl, 4)
for keys in dic.keys():
    dic[keys] = round(dic[keys] / N_sim, 4)
print(dic)
key_max = max(dic, key=dic.get)
print('best chester:', key_max, dic[key_max])

{10: 4.6329, 80: 4.0387, 37: 4.0259, 17: 5.869, 31: 4.4299, 90: 4.0057, 50: 3.8995, 20: 4.1702, 73: 3.9819, 89: 4.0884}
best chester: 17 5.869
